# GHDL memory inference

To infer various types of memory using the GHDL plugin and compare (possibly) against reference verilog implementations, we author the statements that are normally recognized by a synthesizer in MyHDL to be able to emit both VHDL and Verilog.

This involves some more complex tests and is regarded unstable, due to deficiencies in the yosys memory modelling primitives.

**Note** The True dual port inference test requires a modified techmap wrapper from the `bram_ecp5` branch. This will be revisited with the planned [TDP changes](https://github.com/YosysHQ/yosys/issues/1959).

Also, the yosys branch needs to be able to possibly infer the techmap according to the `IMPLEMENTATION` mode, see below.

To simulate against an inferred vendor model **post technology mapping**, you need to install the matching vendor simulation files yourself in this [directory](/tree/work/tests/lib/techlibs/ecp5) and edit [cells_sim.v](/edit/work/tests/lib/techlibs/ecp5/cells_sim.v).

In [1]:
import os
import sys

sys.path.append("ram")

Generate the BRAM mapping verilog headers:

In [2]:
! cd lib && make

python3 techlibs/ecp5/brams_connect.py
python3 techlibs/ecp5/brams_init.py


In [3]:
! ls lib/techlibs/ecp5

DP16KD.v	 bram_conn_36.vh       brams_connect.py  cells_map.v
MULT18X18D.v	 bram_conn_4.vh        brams_init.py	 cells_sim.v
bram_conn_1.vh	 bram_conn_9.vh        brams_map.v	 gsr_pur_assign.v
bram_conn_18.vh  bram_init_1_2_4.vh    cells_ff.vh
bram_conn_2.vh	 bram_init_9_18_36.vh  cells_io.vh


Clean up:

In [4]:
rm -f *.v *.ps *vcd

In [5]:
"""
Dual port RAM test suite

(c) 2020   <hackfin@section5.ch>

LICENSE: GPL v2

See ramgen for configuration of 'IMPLEMENTED' variable

"""

from ram import ramgen
from myhdl import *

# This sets the mapping implementation operation mode.

# IMPLEMENTED = ramgen.IMPLEMENTATION_VHDL_YOSYS_MAPPED # GHDL mapping

IMPLEMENTED = ramgen.IMPLEMENTATION_VHDL_CUSTOM_MAPPED # Custom mapping


@block
def dpram_r1w1_verify(a, b):
    "Verification TB, read before write"
    @instance
    def stim():
        a.ce.next = 1
        b.ce.next = 1

        for i in range(4):
            yield a.clk.posedge

        for i in range(2 ** len(a.addr)):
            yield a.clk.posedge
            a.addr.next = i
            a.write.next = 0xface
            yield a.clk.negedge
            a.we.next = 1
            yield a.clk.negedge
            a.we.next = 0


            b.addr.next = i
            # Data is ready on port B:
            yield b.clk.posedge
            yield b.clk.posedge
            if b.read != 0xface:
                raise ValueError("Mismatch B / 1")

            yield a.clk.posedge
            a.addr.next = i
            a.write.next = 0xdead
            yield a.clk.negedge
            a.we.next = 1
            yield a.clk.negedge
            a.we.next = 0

            b.addr.next = i
            # Data is ready on port B:
            yield b.clk.posedge
            yield b.clk.posedge
            if b.read != 0xdead:
                raise ValueError("Mismatch B / 2")


        print("Simulation Done")

    return instances()

# Make sure to use a minimum of 7 address bits to map to a
# DP16KD primitive (the custom mapper always does, regardless of bit size)

ADDRBITS = 7

ramgen.convert(ramgen.dpram_r1w1, IMPLEMENTED, ADDRBITS=ADDRBITS, trace=True)


/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_ce
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_read
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_sel
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_we
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_ce
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_write
  category=ToVHDLWarning
/hom

Some files were created in the conversion process:

In [6]:
ls -l *.v *.vhd

-rw-r--r-- 1 pyosys users  871 Jun 30 13:57 dpram_r1w1.v
-rw-r--r-- 1 pyosys users 1410 Jun 30 13:57 dpram_r1w1.vhd
-rw-r--r-- 1 pyosys users 3518 Jun 30 13:56 dpram_r2w1_hl.vhd
-rw-r--r-- 1 pyosys users 1681 Jun 30 13:56 dpram_r2w1_raw.vhd
-rw-r--r-- 1 pyosys users 1487 Jun 30 13:56 dpram_r2w1_wt.vhd
-rw-r--r-- 1 pyosys users 4349 Jun 30 13:57 pck_myhdl_011.vhd
-rw-r--r-- 1 pyosys users  922 Jun 30 13:57 tb_dpram_r1w1.v
-rw-r--r-- 1 pyosys users  876 Jun 30 13:57 tb_test_dpram_r1w1.v
-rw-r--r-- 1 pyosys users  923 Jun 30 13:57 test_dpram_r1w1.v
-rw-r--r-- 1 pyosys users 1534 Jun 30 13:57 test_dpram_r1w1.vhd
-rw-r--r-- 1 pyosys users 4828 Jun 30 13:56 test_dpram_r2w1_hl.vhd
-rw-r--r-- 1 pyosys users 1974 Jun 30 13:56 test_dpram_r2w1_raw.vhd
-rw-r--r-- 1 pyosys users 1680 Jun 30 13:56 test_dpram_r2w1_wt.vhd


## Running the memory test bench

You might want to edit the file [config.py](config.py).

This is a complicated test bench, that, depending on the MODE parameter, creates a cosimulation element of an inferred memory. This can be run through a custom mapper.

**Requires installed vendor models**

In [7]:
ramgen.testbench(ramgen.dpram_r1w1, dpram_r1w1_verify, MODE=IMPLEMENTED, ADDRBITS=ADDRBITS)

Running YOSYS custom mapper
Running command 'ghdl dpram_r1w1.vhd pck_myhdl_011.vhd -e dpram_r1w1'
--------------------
Found memory
(IdString "\WR_CLK_POLARITY", Const "1")
(IdString "\WR_CLK_ENABLE", Const "1")
(IdString "\RD_TRANSPARENT", Const "0")
(IdString "\RD_CLK_POLARITY", Const "1")
(IdString "\RD_CLK_ENABLE", Const "1")
(IdString "\RD_PORTS", Const "00000000000000000000000000000001")
(IdString "\WR_PORTS", Const "00000000000000000000000000000001")
(IdString "\ABITS", Const "00000000000000000000000000000111")
(IdString "\SIZE", Const "00000000000000000000000010000000")
(IdString "\OFFSET", Const "00000000000000000000000000000000")
(IdString "\WIDTH", Const "00000000000000000000000000010000")
(IdString "\MEMID", Const "010111000011001000111000")
----------------------------------------
Processing Const "010111000011001000111000"
Instance has 1 write, 1 read ports
RD_CLK [SigSpec at 0x1643d70]
WR_CLK [SigSpec at 0x152d450]
Map <Port A RW clkid=2> to port[\b_clk::wire_1_1]
Map <P

<class 'myhdl._SuspendSimulation'>: Simulated 2000 timesteps


Download created RTL schematic: [dpram_r1w1_mapped.ps](dpram_r1w1_mapped.ps) or the simulation trace [dpram_tb.vcd](dpram_tb.vcd) or the internal trace [dpram_r1w1.vcd](dpram_r1w1.vcd).

## VHDL mapping

Likewise, we use the GHDL plugin to infer the RAM description into a technology-mapped element.
Note that `ADDRBITS` matter for the inferred memory element type. The memory inference is left to yosys.

**Currently this scenario fails**

In [8]:
# IMPLEMENTED = ramgen.IMPLEMENTATION_VERILOG_MODEL # Cosim against verilog model

IMPLEMENTED = ramgen.IMPLEMENTATION_VHDL_YOSYS_MAPPED # GHDL mapping


ADDRBITS = 7
ramgen.convert(ramgen.dpram_r1w1, IMPLEMENTED, ADDRBITS=ADDRBITS, trace=True)
ramgen.testbench(ramgen.dpram_r1w1, dpram_r1w1_verify, MODE=IMPLEMENTED, ADDRBITS=ADDRBITS)

['iverilog', '-s', 'tb_dpram_r1w1', '-D', 'mixed_hdl', '-o', 'dpram_r1w1_mapped.o', 'dpram_r1w1_mapped.v', 'tb_dpram_r1w1.v', 'lib/techlibs/ecp5/cells_sim.v', '-I', 'lib/techlibs/ecp5', 'lib/techlibs/ecp5/gsr_pur_assign.v']


/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_ce
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_read
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_sel
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_we
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_ce
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_write
  category=ToVHDLWarning
/hom

ValueError: Mismatch B / 1

In [9]:
!ls *.vcd


dpram_r1w1.vcd	dpram_tb.1593525452.8778656.vcd  dpram_tb.vcd


### Read-after-write r2 w1

Two read ports, one write port, 'read-after'write' behaviour:

In [10]:
from dpram_r2w1_raw import dpram_r2w1_raw, dpram_r2w1_raw_verify, IMPLEMENTED

ramgen.convert(dpram_r2w1_raw, IMPLEMENTED, ADDRBITS=ADDRBITS, trace=True)
ramgen.testbench(dpram_r2w1_raw, dpram_r2w1_raw_verify, MODE=IMPLEMENTED, ADDRBITS=ADDRBITS)

Simulation Done


/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_sel
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_we
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_write
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_sel
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVerilog.py:312: ToVerilogWarning: Port is not used: a_sel
  category=ToVerilogWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVerilog.py:312: ToVerilogWarning: Port is not used: b_we
  category=ToV

### Variant with H/L select

The `dpram_r2w1_hl` is simple a composed dual read port RAM which uses the `.sel` port pins to select high/low byte for selective byte writes.

In [11]:
from dpram_r2w1_hl import dpram_r2w1_hl, dpram_r2w1_hl_verify, IMPLEMENTED

ramgen.convert(dpram_r2w1_hl, IMPLEMENTED, ADDRBITS=ADDRBITS, trace=True)
ramgen.testbench(dpram_r2w1_hl, dpram_r2w1_hl_verify, MODE=IMPLEMENTED, ADDRBITS=ADDRBITS)

Running YOSYS custom mapper
Running command 'ghdl dpram_r2w1_hl.vhd pck_myhdl_011.vhd -e dpram_r2w1_hl'
--------------------
Found memory
(IdString "\WR_CLK_POLARITY", Const "1")
(IdString "\WR_CLK_ENABLE", Const "1")
(IdString "\RD_TRANSPARENT", Const "00")
(IdString "\RD_CLK_POLARITY", Const "11")
(IdString "\RD_CLK_ENABLE", Const "11")
(IdString "\RD_PORTS", Const "00000000000000000000000000000010")
(IdString "\WR_PORTS", Const "00000000000000000000000000000001")
(IdString "\ABITS", Const "00000000000000000000000000000111")
(IdString "\SIZE", Const "00000000000000000000000010000000")
(IdString "\OFFSET", Const "00000000000000000000000000000000")
(IdString "\WIDTH", Const "00000000000000000000000000001000")
(IdString "\MEMID", Const "010111000011100100110001")
----------------------------------------
--------------------
Found memory
(IdString "\WR_CLK_POLARITY", Const "1")
(IdString "\WR_CLK_ENABLE", Const "1")
(IdString "\RD_TRANSPARENT", Const "00")
(IdString "\RD_CLK_POLARITY", C

/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_ce
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_ce
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:495: ToVHDLWarning: Signal is driven but not read: dpram_r2w1_hl_0_bh_we
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:495: ToVHDLWarning: Signal is driven but not read: dpram_r2w1_hl_0_bl_we
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:495: ToVHDLWarning: Signal is driven but not read: dpram_r2w1_hl_0_bh_write
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/convers

### Write-Through variant

In [12]:
from dpram_r2w1_wt import dpram_r2w1_verify, IMPLEMENTED

rammodel = ramgen.dpram_r2w1_wt
# Verify that this model fails with the test by enabling this line:
# rammodel = dpram_r2w1_raw

ramgen.convert(rammodel, IMPLEMENTED, ADDRBITS=ADDRBITS, trace=True)
ramgen.testbench(rammodel, dpram_r2w1_verify, MODE=IMPLEMENTED, ADDRBITS=ADDRBITS)

['iverilog', '-s', 'tb_dpram_r2w1_wt', '-D', 'mixed_hdl', '-o', 'dpram_r2w1_wt.o', 'dpram_r2w1_wt.v', 'tb_dpram_r2w1_wt.v', 'lib/techlibs/ecp5/cells_sim.v', '-I', 'lib/techlibs/ecp5', 'lib/techlibs/ecp5/gsr_pur_assign.v']
Simulation Done


/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_ce
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: a_sel
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_we
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_ce
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_write
  category=ToVHDLWarning
/home/pyosys/.local/lib/python3.7/site-packages/myhdl-0.11-py3.7.egg/myhdl/conversion/_toVHDL.py:408: ToVHDLWarning: Port is not used: b_sel
  category=ToVHDLWarning
/home